In [1]:
import os
import pandas as pd
import numpy as np

path = '/Users/yuanzh/Google Drive/Files_PNC_Analysis'

# load subject info
filename = 'SubInfo_2018.txt'
sub = pd.read_csv(os.path.join(path, filename), header=0, delimiter='\t')
print("sub shape: " + str(sub.shape))
print("The first 3 columns are: " + str(sub.columns[0:3]))
sub.head(3)

num_sub = sub.shape[0]

sub shape: (759, 10)
The first 3 columns are: Index(['Subj', 'Age', 'Sex'], dtype='object')


In [2]:
# emotion condition (classes)
emotion = ['fear','anger','sad','happy'] # this is a list
#print(emotion)

# load connectivity into dictionary conn
path = '/Users/yuanzh/Google Drive/Files_PNC_Analysis'
path = path +'/BS_Conn_50ROIs_2018/Original Data'
#print(path)

df = {} # save connectivity under Fear, Anger, Sad, and Happy

for i in range(len(emotion)):
    filename = 'bs_conns_' + emotion[i] + '.txt'  
    #print(filename)
    conn = pd.read_csv(os.path.join(path, filename), header=0, delimiter='\t')
    # check the shape
    #print(conn.shape)
    # get name of the last unused column
    col = conn.columns[-1]
    #print("last column: " + str(col))
    # drop the last unused column
    conn = conn.drop(str(col),1)
    # check the shape
    #print(conn.shape)
    df[str(emotion[i])] = pd.concat([sub, conn], axis=1) #concat
    

In [3]:
# Data check
print(df.keys())
print(df[str(emotion[0])].shape)
df[str(emotion[0])].head(3)

dict_keys(['fear', 'anger', 'sad', 'happy'])
(759, 1236)


,Subj,Age,Sex,Race,GScore,GGroup,DScore,DGroup,Motion,Agebin,...,VLPFC.R_DLPFC.L,VLPFC.R_DLPFC.R,VLPFC.R_DLPFCm.L,VLPFC.R_DLPFCm.R,DLPFC.L_DLPFC.R,DLPFC.L_DLPFCm.L,DLPFC.L_DLPFCm.R,DLPFC.R_DLPFCm.L,DLPFC.R_DLPFCm.R,DLPFCm.L_DLPFCm.R
0,600009963128,9.666667,0,0,0,0,0,0,0.149664,Children,...,0.4722,0.9470,0.7317,1.2809,0.8010,0.0175,0.1136,0.2156,0.5137,1.2603
1,600018902293,15.583333,0,0,12,1,4,1,0.059930,Adolescents,...,0.3621,0.3730,0.5669,0.5315,1.1184,1.1773,1.0380,1.1589,0.6858,1.2571
2,600020364885,17.750000,1,0,2,1,0,0,0.056563,Adolescents,...,0.7979,0.4853,0.3248,0.6449,1.0761,0.0941,0.2069,-0.0811,0.1089,0.9859


In [4]:
# combine four emotion data
data = pd.concat([df["fear"],df["anger"],df["sad"],df["happy"]], axis=0)
print(data.shape)

(3036, 1236)


In [5]:
# replace missing values with column mean
def replace_nan(X):
    col_mean = np.nanmean(X, axis=0) # obtain column means
    ind = np.where(np.isnan(X)) # find the position of missing values
    X[ind] = np.take(col_mean, ind[1]) # replace nans with colmeans
    
    return X

In [6]:
# Prepare data for emotion classification

# Features
X = data.drop(columns = sub.columns) # drop subID, age, group etc.
#X.head(3)
X = X.drop(columns = X.columns[0]) # drop subID
#X.head(3)
X = X.values # convert pandas dataframe to numpy array
# print(X.shape)
#print(type(X))
X = replace_nan(X) # deal with missing values in numpy array

# Labels
y = np.array([1,2,3,4]) 
y = np.repeat(y, num_sub)

# Groups
# Group info that will be used to do train-test split
# data belongs to the same subject should not be splited into different folds
G = np.array(range(num_sub)) +1
G = np.tile(G, 4).T
#print(G[0,759:789])
print('X shape: ' + str(X.shape) + '\ty shape: ' + str(y.shape) + '\tG shape: ' + str(G.shape))


X shape: (3036, 1225)	y shape: (3036,)	G shape: (3036,)


In [7]:
# import packages for svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GroupShuffleSplit

In [33]:
# train-test split for 10 fold cross validation; 
# samples that belong to the same subject will not be
# split into different folds
def get_cv(X, y, G):
    cv = GroupShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 42)
    nfold = cv.get_n_splits()
    cv = cv.split(X, y, G)
    
    return cv, nfold

# train_inds, test_inds = next(cv) 

In [ ]:
"""
# Example of subsetting numpy array
a = np.arange(100).reshape(10,10)
#print(a)
n1, n2 = np.arange(5), np.arange(5)
#print(n1.shape)
b = a[n1[:,None],n2[None,:]]
b

n1 = np.concatenate((np.arange(100), np.arange(100)+759))
n1
"""

In [ ]:
"""
# Test using glmnet
import glmnet

model = glmnet.logistic.LogitNet(alpha=0.2)

inds = np.concatenate((np.arange(100), np.arange(100)+759, np.arange(100)+759*2, np.arange(100)+759*3))
m = model.fit(X[inds,:],y[inds])

m.predict(X[inds+100,:]) # this returns the predicted value at the best lambda
np.sum(m.predict == y[inds+100])
m.lambda_best_  # best lambda
m.lambda_best_inx_  # the position of the best lambda
"""

In [14]:
a = np.array([1,1,0,0,1,1,0,0,])
b = np.array([0,1,0,1,0,1,0,1])

c = []
c = np.concatenate((c,a))
c = np.concatenate((c, b))
c

array([1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.])

In [35]:
import glmnet

alpha_range = (np.arange(10)+1)*0.1
results = []

cv_splits, nfold = get_cv(X, y, G)
print("number of folds: " + str(nfold))
# get train and test indices for all folds
train_folds = []
test_folds = []

for train_inds, test_inds in cv_splits:
    train_folds.append(train_inds)
    test_folds.append(test_inds)


    
train_preds = []
test_preds = []

for alpha in alpha_range:
    print("alpha " + str(alpha))
    
    # nfold cross validation
    for fd in range(nfold):
    #for train_inds, test_inds in cv_splits:
        print("fold " + str(fd))
        X_train, y_train, X_test, y_test = X[train_folds[fd],:], y[train_folds[fd]], X[test_folds[fd],:], y[test_folds[fd]]
        
        model = glmnet.logistic.LogitNet(alpha=alpha)
        m = model.fit(X_train,y_train)
        train_preds = np.concatenate((train_preds, m.predict(X_train) == y_train))
        test_preds = np.concatenate((test_preds, m.predict(X_test) == y_test))
        results.append((alpha, m.lambda_best_, train_preds, test_preds, m)) 
        print("training accuracy: " + str(np.mean(train_preds)) + " at alpha: " + str(alpha) + " and lambda: " + str(m.lambda_best_))
        print("test accuracy " + str(np.mean(test_preds)) + " at alpha: " + str(alpha) + " and lambda: " + str(m.lambda_best_)) 
        fd += 1
    
    train_acc = np.mean(train_preds)
    test_acc = np.mean(test_preds)
    print("training accuracy: " + str(train_acc) + " at alpha: " + str(alpha))
    print("test accuracy " + str(test_acc) + " at alpha: " + str(alpha))  
 

number of folds: 10
alpha 0.1
fold 0
training accuracy: 0.42174629324546953 at alpha: 0.1 and lambda: [0.13207011]
test accuracy 0.3157894736842105 at alpha: 0.1 and lambda: [0.13207011]
fold 1
training accuracy: 0.39847611202635913 at alpha: 0.1 and lambda: [0.18459305]
test accuracy 0.3207236842105263 at alpha: 0.1 and lambda: [0.18459305]
fold 2
training accuracy: 0.3938769906644701 at alpha: 0.1 and lambda: [0.17067644]
test accuracy 0.3157894736842105 at alpha: 0.1 and lambda: [0.17067644]
fold 3
training accuracy: 0.3902388797364086 at alpha: 0.1 and lambda: [0.17758013]
test accuracy 0.31085526315789475 at alpha: 0.1 and lambda: [0.17758013]
fold 4
training accuracy: 0.41046128500823725 at alpha: 0.1 and lambda: [0.06596863]
test accuracy 0.31282894736842104 at alpha: 0.1 and lambda: [0.06596863]
fold 5
training accuracy: 0.40266337177375067 at alpha: 0.1 and lambda: [0.17651345]
test accuracy 0.3149671052631579 at alpha: 0.1 and lambda: [0.17651345]
fold 6
training accuracy: 0.

training accuracy: 0.40207779981844466 at alpha: 0.5 and lambda: [0.02457028]
test accuracy 0.30796858216971 at alpha: 0.5 and lambda: [0.02457028]
fold 9
training accuracy: 0.40454695222405274 at alpha: 0.5 and lambda: [0.01213584]
test accuracy 0.30838815789473684 at alpha: 0.5 and lambda: [0.01213584]
training accuracy: 0.40454695222405274 at alpha: 0.5
test accuracy 0.30838815789473684 at alpha: 0.5
alpha 0.6000000000000001
fold 0
training accuracy: 0.404738831282101 at alpha: 0.6000000000000001 and lambda: [0.0241578]
test accuracy 0.30837203302373584 at alpha: 0.6000000000000001 and lambda: [0.0241578]
fold 1
training accuracy: 0.4042817767076416 at alpha: 0.6000000000000001 and lambda: [0.03076551]
test accuracy 0.3085463056680162 at alpha: 0.6000000000000001 and lambda: [0.03076551]
fold 2
training accuracy: 0.40339125299182493 at alpha: 0.6000000000000001 and lambda: [0.0342634]
test accuracy 0.30821747765640517 at alpha: 0.6000000000000001 and lambda: [0.0342634]
fold 3
train

training accuracy: 0.4048862560902941 at alpha: 1.0 and lambda: [0.01343329]
test accuracy 0.3091580347144457 at alpha: 1.0 and lambda: [0.01343329]
fold 4
training accuracy: 0.4056143241134137 at alpha: 1.0 and lambda: [0.00872067]
test accuracy 0.3092797783933518 at alpha: 1.0 and lambda: [0.00872067]
fold 5
training accuracy: 0.4050701880834706 at alpha: 1.0 and lambda: [0.01937233]
test accuracy 0.3091419956140351 at alpha: 1.0 and lambda: [0.01937233]
fold 6
training accuracy: 0.40491091900337983 at alpha: 1.0 and lambda: [0.01685752]
test accuracy 0.3091257460661964 at alpha: 1.0 and lambda: [0.01685752]
fold 7
training accuracy: 0.4053516793867465 at alpha: 1.0 and lambda: [0.0112324]
test accuracy 0.3090426960257787 at alpha: 1.0 and lambda: [0.0112324]
fold 8
training accuracy: 0.4052593480105836 at alpha: 1.0 and lambda: [0.01479749]
test accuracy 0.30912745879851145 at alpha: 1.0 and lambda: [0.01479749]
fold 9
training accuracy: 0.4061079077429984 at alpha: 1.0 and lambda: 

In [ ]:
"""
import glmnet
# define my glmnet model (data normalization, glmnet)
def myGLMNet(alpha):
    pipe = Pipeline([
        ("std_scaler", StandardScaler()),
        ("glmnet", glmnet.logistic.LogitNet(alpha = alpha).fit())
    ])

# define evaluation
def evaluation(X, y, G, alpha):
    pipe = myGLMNet(alpha=alpha) 
    cv = get_cv(X, y, G)
        
    results = cross_validate(pipe, X, y, scoring=['accuracy'], cv=cv, verbose=1, return_train_score=True, n_jobs=-1)
    
    return results
    
alpha_range = [0.1,0.5] #np.logspace(-7, 1, 9)
models = []
for alpha in alpha_range:
        results = evaluation(X,y,G,alpha)
        models.append((alpha,results))
        print("Training score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['train_accuracy']),np.std(results['train_accuracy'])))
        print("Validation score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']),np.std(results['test_accuracy'])))
"""

In [ ]:
"""
# define my SVC model (data normalization, SVC)
def mySVC(gamma, C):
    return Pipeline([
        ("std_scaler", StandardScaler()),
        ("svc", SVC(kernel="rbf", gamma=gamma, C=C))
    ])
"""

In [ ]:
# model = mySVC(gamma=1, C=0.1)
# model.fit(X_train,y_train)
# model.score(X_test, y_test)

In [ ]:
"""
def evaluation(X, y, G, gamma, C):
    pipe = mySVC(gamma=gamma, C=C) 
    cv = get_cv(X, y, G)
    #print("type(cv) = {}".format(type(cv)))
    
    results = cross_validate(pipe, X, y, scoring=['accuracy'], cv=cv, verbose=1, return_train_score=True, n_jobs=-1)
    
    return results
"""

In [ ]:
#results = evaluation(X,y,G,gamma=1,C=0.1)
#print("Training score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['train_accuracy']),
#                                                         np.std(results['train_accuracy'])))
#print("Validation score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']),
#                                                           np.std(results['test_accuracy'])))

In [ ]:
"""
C_range = np.exp2(np.array(range(-5,5))) #np.logspace(-3, 2, 6)
gamma_range = np.exp2(np.array(range(-10,1)))
models = []
for C in C_range:
    for gamma in gamma_range:
        results = evaluation(X,y,G,gamma,C)
        models.append((C,gamma,results))
        print("Training score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['train_accuracy']),
                                                         np.std(results['train_accuracy'])))
        print("Validation score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']),
                                                           np.std(results['test_accuracy'])))
"""

In [ ]:
"""
# GridSearch
from sklearn.model_selection import GridSearchCV

# set the parameters
#tuned_parameters = [{'kernal': ['rbf'], 'gamma':[np.exp2(np.array(range(-2,2)))], 'C': [np.exp2(np.array(range(-2,2)))]}]

estimators = [("std_scaler", StandardScaler()), ("svc", SVC())]
tuned_parameters = [{'svc__gamma':[np.exp2(np.array(range(-2,2)))], 'svc__C': [np.exp2(np.array(range(-2,2)))]}]


pipe = Pipeline(estimators)
#cv = get_cv(X, y, G)
clf = GridSearchCV(pipe, param_grid = tuned_parameters, cv = 4, scoring='accuracy', n_jobs=-1)
clf.fit(X,y)

print('Best parameters set found: ')
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

"""